In [1]:
import pandas as pd
import numpy as np
import re
import gensim
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
from wordcloud import WordCloud
from gensim import corpora, models, similarities
import pyLDAvis
from pprint import pprint
import pickle 
import os
import matplotlib.pyplot as plt
from gensim.models import CoherenceModel
from gensim.test.utils import datapath
import random
from gensim.models.ldamodel import LdaModel
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from gensim.corpora import MmCorpus
import mymodule

In [2]:
data = pd.read_csv('dataset/DatasetLegal.csv')
str_answer = data['answer'].astype(str)
str_answer = str_answer.map(lambda x: re.sub('[,.!?#/]', '', x))
str_question = data['question'].astype(str)
str_question = str_question.map(lambda x: re.sub('[,.!?*/]', '', x))

In [3]:
sentense_token = []
for i in range(len(str_answer)):
  sentense_token.append(str_question[i])
  sentense_token.append(str_answer[i])

train_data = sentense_token[:13062]
test_data = sentense_token[13062:]

In [4]:
word_token_answer = []
for sentense in train_data:
  word = word_tokenize(sentense, engine='newmm')
  word_token_answer.append(word)

In [5]:
stopwords = list(thai_stopwords())
read_stopwords = pd.read_csv('dataset/add_stopwords.csv')
add_stopwords = read_stopwords['stopword'].values.tolist()
processed_answer = []
for sentense in word_token_answer:
  each_sentense = []
  for word in sentense:
    if(word not in stopwords + add_stopwords):
      each_sentense.append(word)
  processed_answer.append(each_sentense)
print(processed_answer[0][:8])

['มีเรื่อง', 'ปรึกษา', 'ครอบครัว', 'สินสมรส', 'สินส่วนตัว', 'การทราบ', 'ข้อเท็จจริง', 'วิธีการ']


In [6]:
id2word = corpora.Dictionary(processed_answer)
# print(id2word)
with open('model/id2word.pkl', 'wb') as f:
    pickle.dump(id2word, f)

2023-04-26 22:30:21.845 INFO    gensim.corpora.dictionary: adding document #0 to Dictionary<0 unique tokens: []>
2023-04-26 22:30:23.288 INFO    gensim.corpora.dictionary: adding document #10000 to Dictionary<14634 unique tokens: ['กฎหมาย', 'การทราบ', 'การร้องขอ', 'ขอโทษ', 'ข้อตกลง']...>
2023-04-26 22:30:23.759 INFO    gensim.corpora.dictionary: built Dictionary<16508 unique tokens: ['กฎหมาย', 'การทราบ', 'การร้องขอ', 'ขอโทษ', 'ข้อตกลง']...> from 13062 documents (total 655234 corpus positions)
2023-04-26 22:30:23.763 INFO    gensim.utils: Dictionary lifecycle event {'msg': "built Dictionary<16508 unique tokens: ['กฎหมาย', 'การทราบ', 'การร้องขอ', 'ขอโทษ', 'ข้อตกลง']...> from 13062 documents (total 655234 corpus positions)", 'datetime': '2023-04-26T22:30:23.763183', 'gensim': '4.3.0', 'python': '3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'created'}


In [7]:
corpus = []
for text in processed_answer:
  vec = id2word.doc2bow(text)
  corpus.append(vec)

In [8]:
num_topics = 6
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       iterations=100,
                                       chunksize=2000,
                                       passes=10,
                                       alpha=0.9,
                                       eta=0.5
                                       )

2023-04-26 22:30:25.194 INFO    gensim.models.ldamodel: using serial LDA version on this node
2023-04-26 22:30:25.239 INFO    gensim.models.ldamodel: running online (multi-pass) LDA training, 6 topics, 10 passes over the supplied corpus of 13062 documents, updating model once every 2000 documents, evaluating perplexity every 13062 documents, iterating 100x with a convergence threshold of 0.001000
2023-04-26 22:30:25.241 INFO    gensim.models.ldamodel: PROGRESS: pass 0, at document #2000/13062
2023-04-26 22:30:34.131 INFO    gensim.models.ldamodel: merging changes from 2000 documents into a model of 13062 documents
2023-04-26 22:30:34.161 INFO    gensim.models.ldamodel: topic #2 (0.900): 0.012*"เงิน" + 0.011*"บริษัท" + 0.010*"มาตรา" + 0.009*"ฟ้อง" + 0.008*"ศาล" + 0.007*"จ่าย" + 0.007*"บ้าน" + 0.006*"กรณี" + 0.006*"สิทธิ" + 0.005*"รถ"
2023-04-26 22:30:34.164 INFO    gensim.models.ldamodel: topic #5 (0.900): 0.012*"ศาล" + 0.008*"มาตรา" + 0.008*"ฟ้อง" + 0.007*"กรณี" + 0.007*"ที่ดิน" + 0.00

In [19]:
with open('model/lda_model.pkl', 'wb') as f:
    pickle.dump(lda_model, f)
with open('model/corpus.mm', 'wb') as f:
    pickle.dump(corpus, f)

In [10]:
lda_model.show_topics(num_topics=6, num_words=10)

[(0,
  '0.023*"ศาล" + 0.021*"มาตรา" + 0.016*"ฟ้อง" + 0.015*"สัญญา" + 0.013*"เจ้าหนี้" + 0.013*"ชำระหนี้" + 0.012*"จำเลย" + 0.010*"สิทธิ" + 0.010*"ลูกหนี้" + 0.010*"บังคับ"'),
 (1,
  '0.037*"นายจ้าง" + 0.031*"ลูกจ้าง" + 0.029*"บริษัท" + 0.023*"ทำงาน" + 0.017*"จ่าย" + 0.015*"พนักงาน" + 0.014*"งาน" + 0.013*"สิทธิ" + 0.013*"เลิกจ้าง" + 0.011*"กรณี"'),
 (2,
  '0.025*"เงิน" + 0.020*"รถ" + 0.018*"จ่าย" + 0.017*"บาท" + 0.012*"บริษัท" + 0.010*"แจ้ง" + 0.009*"คืน" + 0.009*"ค่า" + 0.009*"ธนาคาร" + 0.008*"สัญญา"'),
 (3,
  '0.052*"ที่ดิน" + 0.040*"บ้าน" + 0.028*"แม่" + 0.026*"พ่อ" + 0.021*"ลูก" + 0.019*"โอน" + 0.014*"ชื่อ" + 0.012*"ขาย" + 0.010*"แบ่ง" + 0.009*"เสียชีวิต"'),
 (4,
  '0.018*"ศาล" + 0.017*"แฟน" + 0.016*"ตำรวจ" + 0.012*"โดน" + 0.010*"จำคุก" + 0.008*"รอ" + 0.008*"ทนาย" + 0.007*"ข้อหา" + 0.007*"ทราบ" + 0.006*"แจ้งความ"'),
 (5,
  '0.051*"สามี" + 0.039*"บุตร" + 0.025*"มาตรา" + 0.021*"สิทธิ" + 0.019*"บิดา" + 0.018*"ภรรยา" + 0.015*"ปพพ" + 0.013*"มารดา" + 0.012*"เด็ก" + 0.012*"ทายาท"')]

In [11]:
import pickle
topic_dict = {
    0 : "criminal",
    1 : "labor",
    2 : "contract",
    3 : "succession",
    4 : "violation",
    5 : "family",
}

with open('model/topic_dict.pkl', 'wb') as f:
    pickle.dump(topic_dict, f)


In [12]:
# import pyLDAvis.gensim_models as gensimvis
# num_topics = 6
# with open('model/id2word.pkl', 'rb') as f:
#   id2word = pickle.load(f)
# with open('model/lda_model.pkl', 'rb') as f:
#   lda_model = pickle.load(f)
# pyLDAvis.enable_notebook()
# LDAvis_data_filepath = os.path.join('./ldavis_prepared_'+str(num_topics))

# LDAvis_data_filepath
# if 1 == 1:
#     LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
#     with open(LDAvis_data_filepath, 'wb') as f:
#         pickle.dump(LDAvis_prepared, f)
# with open(LDAvis_data_filepath, 'rb') as f:
#     LDAvis_prepared = pickle.load(f)
# pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared_'+ str(num_topics) +'.html')
# LDAvis_prepared

The sections below working on lda model in question part

In [12]:
sentense_token_question = []
# for each_question in str_question:
#   sentense_token_question.append(each_question)
for i in range(6531):
  sentense_token_question.append(str_question[i])
# print(len(question_data))
topic_question = []
corpus_question = []
for sentense in sentense_token_question:
  processed_question = mymodule.preprocess(sentense)
  corpus_question.append(processed_question)
  topic = lda_model.get_document_topics(processed_question)
  topic_question.append(topic)

In [13]:
with open('model/question_lda.pkl', 'wb') as f:
    pickle.dump(topic_question, f)
MmCorpus.serialize('model/corpus_question.mm', corpus_question)

2023-04-26 22:38:19.285 INFO    gensim.corpora.mmcorpus: storing corpus in Matrix Market format to model/corpus_question.mm
2023-04-26 22:38:19.289 INFO    gensim.matutils: saving sparse matrix to model/corpus_question.mm
2023-04-26 22:38:19.292 INFO    gensim.matutils: PROGRESS: saving document #0
2023-04-26 22:38:19.446 INFO    gensim.matutils: PROGRESS: saving document #1000
2023-04-26 22:38:19.586 INFO    gensim.matutils: PROGRESS: saving document #2000
2023-04-26 22:38:19.744 INFO    gensim.matutils: PROGRESS: saving document #3000
2023-04-26 22:38:19.918 INFO    gensim.matutils: PROGRESS: saving document #4000
2023-04-26 22:38:20.062 INFO    gensim.matutils: PROGRESS: saving document #5000
2023-04-26 22:38:20.230 INFO    gensim.matutils: PROGRESS: saving document #6000
2023-04-26 22:38:20.315 INFO    gensim.matutils: saved 6531x16508 matrix, density=0.217% (233909/107813748)
2023-04-26 22:38:20.317 INFO    gensim.corpora.indexedcorpus: saving MmCorpus index to model/corpus_questi

The sections below are in the process of being updated with new documents.

In [14]:
new_doc = random.choice(test_data)
print(new_doc)
test_doc = mymodule.preprocess(new_doc)

ภรรยาน้อยไม่ใช่ภรรยาโดยชอบด้วยกฏหมายฉนั้นจึงไม่มีสิทธิใดๆในสินสมรสส่วนการแต่งงานนั้นหากเป็นการจดทะเบียนสมรสที่ถูกต้องตามกฏหมายก็ถือได้ว่าเป็นการทำนิติกรรมอย่างนึงที่มุ่งก่อให้เกิดสิทธิต่อกันระหว่างคู่สมรสแต่ถ้าจัดงานเลี้ยงเฉยๆโดยมิได้จดทะเบียนสมรสก็ไม่ถือว่าเป็นสามีภรรยากันโดยชอบด้วยกฏหมายหากแต่เป็นการจัดงานเลี้ยงสังสรรค์กันเท่านั้นเพราะไม่ก่อให้เกิดสิทธิใดๆในทางกฏหมายแต่ก็อาจก่อให้เกิดสิทธิบางประการได้ในฐานะกรรมสิทธิ์ร่วมส่วนเรื่องมรดกหากภรรยาน้อยมิได้เป็นญาติหรือมีส่วนได้เสียกับภรรยาหลวงหรือสามีก็ถือเป็นบุคคลภายนอกที่ไม่เกี่ยวข้องกับทายาทตามลำดับชั้นแต่อย่างใดอีกทั้งก็ไม่ถือเป็นคู่สมรสภายใต้บทบัญญติพิเศษตามมาตราในอันที่จะถือเป็นทายาทได้เพราะไม่ใช่ภรรยาโดยชอบด้วยกฏหมาย


In [15]:
with open('model/question_lda.pkl', 'rb') as f:
    topic_question = pickle.load(f)
with open('model/topic_dict.pkl', 'rb') as f:
    topic_dict = pickle.load(f)
new_doc_topics = lda_model.get_document_topics(test_doc)
new_doc_topics_dict = {topic_dict[topic]: prob for topic, prob in new_doc_topics}
print(new_doc_topics_dict)

{'criminal': 0.027091896, 'labor': 0.025308551, 'contract': 0.12543719, 'succession': 0.11737287, 'violation': 0.026783038, 'family': 0.6780064}


In [ ]:
# print(index)
# print(corpus_lda)
# print(sims)

In [18]:
corpus = corpus
data = pd.read_csv('dataset/DatasetLegal.csv')
with open('model/id2word.pkl', 'rb') as f:
  id2word = pickle.load(f)
corpus_lda = lda_model[corpus]
index = similarities.MatrixSimilarity(corpus_lda, num_features=len(id2word))
sims = index[new_doc_topics]
sims_sorted = sorted(enumerate(sims), key=lambda item: -item[1])
print(f"Topic distribution for new document : {new_doc_topics}\n{new_doc}\n")
for doc_id, similarity in sims_sorted[:5]:
  print(f"Document ID: {doc_id}, Similarity score: {similarity}")
  print(data.answer[doc_id])
  print("Topic distribution for similar document : ")
  for num, dis in corpus_lda[doc_id]:
    print(f"\t({topic_dict.get(num)}, {'%.5f' %dis})")

2023-04-26 22:55:15.009 INFO    gensim.similarities.docsim: creating matrix with 6531 documents and 16508 features


Topic distribution for new document : [(0, 0.027091896), (1, 0.025308551), (2, 0.12543719), (3, 0.11737287), (4, 0.026783038), (5, 0.6780064)]
ภรรยาน้อยไม่ใช่ภรรยาโดยชอบด้วยกฏหมายฉนั้นจึงไม่มีสิทธิใดๆในสินสมรสส่วนการแต่งงานนั้นหากเป็นการจดทะเบียนสมรสที่ถูกต้องตามกฏหมายก็ถือได้ว่าเป็นการทำนิติกรรมอย่างนึงที่มุ่งก่อให้เกิดสิทธิต่อกันระหว่างคู่สมรสแต่ถ้าจัดงานเลี้ยงเฉยๆโดยมิได้จดทะเบียนสมรสก็ไม่ถือว่าเป็นสามีภรรยากันโดยชอบด้วยกฏหมายหากแต่เป็นการจัดงานเลี้ยงสังสรรค์กันเท่านั้นเพราะไม่ก่อให้เกิดสิทธิใดๆในทางกฏหมายแต่ก็อาจก่อให้เกิดสิทธิบางประการได้ในฐานะกรรมสิทธิ์ร่วมส่วนเรื่องมรดกหากภรรยาน้อยมิได้เป็นญาติหรือมีส่วนได้เสียกับภรรยาหลวงหรือสามีก็ถือเป็นบุคคลภายนอกที่ไม่เกี่ยวข้องกับทายาทตามลำดับชั้นแต่อย่างใดอีกทั้งก็ไม่ถือเป็นคู่สมรสภายใต้บทบัญญติพิเศษตามมาตราในอันที่จะถือเป็นทายาทได้เพราะไม่ใช่ภรรยาโดยชอบด้วยกฏหมาย

Document ID: 177, Similarity score: 0.9977664351463318
ตามกฎหมายไทยบุตรย่อมเป็นบุตรโดยชอบด้วยกฎหมายของหญิงที่เป็นมารดาเสมอส่วนกรณีบิดาหากบิดาจดทะเบียนรับรองบุตรจะเป็นที่ชัดเจนไม